In [13]:
!pip install fer opencv-python-headless tensorflow scikit-learn flask pyngrok flask-cors

In [14]:
from flask import Flask, request, jsonify
import cv2, base64, numpy as np
from fer import FER
from pyngrok import ngrok
from flask_cors import CORS

In [15]:
detector = FER(mtcnn=True)

def extract_features(image):
    emotions = detector.detect_emotions(image)
    if not emotions: return None
    emo = emotions[0]['emotions']
    return [emo['angry'], emo['fear'], emo['sad'], emo['happy'], emo['neutral']]

def classify_stress(features):
    stress_score = features[0] + features[1] + features[2]
    fear_score = features[1]
    anxiety_score = features[1] + features[2] + features[4]
    if stress_score > 0.2:
        return "STRESSED"
    elif fear_score > 0.2:
        return "FEARED"
    elif anxiety_score > 0.2:
        return "ANXIETY"
    else:
        return "RELAXED"

app = Flask(__name__)
CORS(app)

In [16]:
@app.route("/detect", methods=["POST"])
def detect():
    data = request.json['image']

    if "," in data:
            data = data.split(",")[1]

    img_data = base64.b64decode(data)
    nparr = np.frombuffer(img_data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    features = extract_features(img)
    if not features:
        return jsonify({"status": "no_face_detected"})

    return jsonify({"status": classify_stress(features), "features": features})

In [ ]:
!pkill ngrok
!ngrok config add-authtoken <Add your ngrok auth token here>

public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app.run(port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://603ae4b56b8a.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:52:48] "OPTIONS /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:52:48] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:52:58] "OPTIONS /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:52:59] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:53:03] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:53:08] "OPTIONS /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:53:09] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:53:13] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:53:18] "OPTIONS /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2025 14:53:19] "POS